In [6]:
import os
import math

depth_dir = "NYUv2/train_depth"
bash_file = "run_all.sh"

num_procs = os.cpu_count()
print("Number of processors:", num_procs)

max_parallel = num_procs

# list all training data and extract image number
files = sorted([
    f[:-4] for f in os.listdir(depth_dir) if f.endswith(".png")
])

num_batches = math.ceil(len(files) / max_parallel)

batch = 0
with open(bash_file, "w") as f:
    f.write("#!/bin/bash\n\n")
    f.write(f"echo 'Starting batch {batch+1}/{num_batches}'\n")

    for i, num in enumerate(files, 1):
        f.write(f"python run_coordinate_descent.py {num} &\n")
        # insert a wait every max_parallel jobs
        if i % max_parallel == 0:
            f.write("wait\n\n")
            batch += 1
            f.write(f"echo 'Starting batch {batch+1}/{num_batches}'\n")

    # final wait for leftover jobs
    f.write("wait\n\n")

print(f"Wrote {bash_file} with {len(files)} jobs in {num_batches} parallel batches.")

Number of processors: 144
Wrote run_all.sh with 795 jobs in 6 parallel batches.
